# Analyze Historical Calendar Events

In [64]:
import os
import sys
import json
import csv
from datetime import datetime
from importlib import import_module

import pandas as pd

sys.path.append(os.path.abspath('../src'))
from gcal.util import get_client

In [70]:
USERNAME = 'zack@plenty.ag'
START_DT = datetime(2016, 6, 13)
END_DT = datetime.utcnow()

START_DT_STR = START_DT.isoformat() + 'Z'
END_DT_STR = END_DT.isoformat() + 'Z'

In [71]:
CACHE_PATH = 'HIST_CACHE.csv'

In [72]:
def load_events():
    if not os.path.exists(CACHE_PATH):
        calendar = get_client()

        all_results = []
        req = calendar.events().list(
            calendarId='zack@plenty.ag',
            timeMin=START_DT_STR,
            timeMax=END_DT_STR,
            singleEvents=True,
            orderBy='startTime'
        )
        while req:
            res = req.execute()
            all_results += res['items']
            req = calendar.events().list_next(req, res)
            print(all_results[-1].get('start', {}).get('dateTime'))

        headers = set()
        for e in all_results:
            headers |= e.keys()

        with open(CACHE_PATH, 'w+') as fout:
            writer = csv.DictWriter(fout, fieldnames=headers)
            writer.writeheader()
            for e in all_results:
                writer.writerow(e)

    return pd.read_csv(CACHE_PATH)

events = load_events()

In [73]:
print((END_DT - START_DT).days)
print(len(events))
print(events.columns)
print(events.loc[0])

1205
4938
Index(['originalStartTime', 'creator', 'description', 'attachments',
       'reminders', 'kind', 'guestsCanInviteOthers', 'conferenceData',
       'extendedProperties', 'guestsCanSeeOtherGuests', 'recurringEventId',
       'created', 'transparency', 'hangoutLink', 'etag', 'status', 'htmlLink',
       'start', 'updated', 'iCalUID', 'id', 'organizer', 'privateCopy', 'end',
       'guestsCanModify', 'attendees', 'location', 'sequence', 'visibility',
       'summary'],
      dtype='object')
originalStartTime                                                        NaN
creator                                    {'email': 'nate@seejanefarm.com'}
description                Hi Guys,\n\nI look forward to our conversation...
attachments                                                              NaN
reminders                                               {'useDefault': True}
kind                                                          calendar#event
guestsCanInviteOthers               

In [68]:
a = events.apply(lambda row: eval(row.get('creator', {})).get('email'), axis=1)


Series([], dtype: object)